# Predicting the Risk of Diabetes in Pregnant Women

Authors: 
- Angela Yang
- Tina Chen
- Tom Cui 
- Yiyang Zhang

### Introduction: ###

Diabetes is a chronic health condition where blood glucose is unable to enter cells due to the lack of insulin facilitating this exchange (NIDDK, 2016). With diabetes being one of the top causes of death in North America (CDC, 2022), the need to determine the best health indicators becomes increasingly important for the correct diagnosis of diabetes. This study uses a dataset from Kaggle, originally from the National Institute of Diabetes and Digestive and Kidney Diseases, showing a variety of health indicators (e.g. blood pressure, Body Mass Index/BMI, glucose, insulin) and whether they were diagnosed with diabetes. 

For our project, we will try to answer this predictive question: Given the health profile of a patient, how accurate can our classification model predict potential diabetes cases based on the selected explanatory variables?

### Methods: ###